In [1]:
import pandas as pd

In [2]:
bayareafips = {
    "06001": "Alameda",
    "06013": "Contra Costa",
    "06041": "Marin",
    "06055": "Napa",
    "06075": "San Francisco",
    "06081": "San Mateo",
    "06085": "Santa Clara",
    "06097": "Sonoma",
    "06095": "Solano",
}

PARCEL_TAZ_CROSSWALK_FILE = r"M:\urban_modeling\baus\BAUS Inputs\basis_inputs\crosswalks\2020_08_17_parcel_to_taz1454sub.csv"
parcel_taz_crosswalk_df = pd.read_csv(PARCEL_TAZ_CROSSWALK_FILE, usecols=['PARCEL_ID', 'ZONE_ID', 'manual_county'])
parcel_taz_crosswalk_df.columns = parcel_taz_crosswalk_df.columns.str.lower()
parcel_taz_crosswalk_df["county"] = parcel_taz_crosswalk_df['manual_county'].map(
    lambda x: f"06{x:03d}"
).map(bayareafips)
del parcel_taz_crosswalk_df['manual_county']
print(f"  Read {len(parcel_taz_crosswalk_df):,} rows from crosswalk {PARCEL_TAZ_CROSSWALK_FILE}")
print(f"  parcel_taz_crosswalk_df.head():\n{parcel_taz_crosswalk_df.head()}")

TAZ_SD_CROSSWALK_FILE = r"M:\urban_modeling\baus\BAUS Inputs\basis_inputs\crosswalks\taz_geography.csv"
taz_sd_crosswalk_df = pd.read_csv(TAZ_SD_CROSSWALK_FILE, usecols=['zone', 'superdistrict'])
taz_sd_crosswalk_df.rename(columns={"zone": "zone_id"}, inplace=True)
print(f"  Read {len(taz_sd_crosswalk_df):,} rows from crosswalk {TAZ_SD_CROSSWALK_FILE}")
print(f"  taz_sd_crosswalk_df.head():\n{taz_sd_crosswalk_df.head()}")

parcel_taz_sd_crosswalk_df = pd.merge(
    left     = parcel_taz_crosswalk_df,
    right    = taz_sd_crosswalk_df,
    on       = 'zone_id',
    how      = 'left',
    validate = 'many_to_one'
)
display(parcel_taz_sd_crosswalk_df.head())

  Read 1,956,208 rows from crosswalk M:\urban_modeling\baus\BAUS Inputs\basis_inputs\crosswalks\2020_08_17_parcel_to_taz1454sub.csv
  parcel_taz_crosswalk_df.head():
   parcel_id  zone_id   county
0     229116      725  Alameda
1     244166      715  Alameda
2     202378      820  Alameda
3    2004420     1401   Sonoma
4     340332      763  Alameda
  Read 1,454 rows from crosswalk M:\urban_modeling\baus\BAUS Inputs\basis_inputs\crosswalks\taz_geography.csv
  taz_sd_crosswalk_df.head():
   zone_id  superdistrict
0        1              1
1        2              1
2        3              1
3        4              1
4        5              1


,parcel_id,zone_id,county,superdistrict
0,229116,725,Alameda,15
1,244166,715,Alameda,15
2,202378,820,Alameda,17
3,2004420,1401,Sonoma,31
4,340332,763,Alameda,16


In [3]:
np_runId = 'PBA50Plus_NoProject_v37'
fbp_runid = 'PBA50Plus_Final_Blueprint_v65'

blg_2020_NP = pd.read_csv(
    r"M:\urban_modeling\baus\PBA50Plus\PBA50Plus_NoProject\{}\core_summaries\{}_building_summary_2020.csv".format(np_runId, np_runId),
    usecols=["parcel_id", 'year_built', 'non_residential_sqft', 'residential_units', 'building_type'],
    dtype={"parcel_id": int}
)
blg_2025_NP = pd.read_csv(
    r"M:\urban_modeling\baus\PBA50Plus\PBA50Plus_NoProject\{}\core_summaries\{}_building_summary_2025.csv".format(np_runId, np_runId),
    usecols=["parcel_id", 'year_built', 'non_residential_sqft', 'residential_units', 'building_type'],
    dtype={"parcel_id": int}
)
blg_2050_FBP = pd.read_csv(
    r"M:\urban_modeling\baus\PBA50Plus\PBA50Plus_FinalBlueprint\{}\core_summaries\{}_building_summary_2050.csv".format(fbp_runid, fbp_runid),
    usecols=["parcel_id", 'year_built', 'non_residential_sqft', 'residential_units', 'building_type'],
    dtype={"parcel_id": int}
)


In [4]:
building_type_dict = {
    'HS': 'single-family',
    'HT': 'single-family',
    'HM': 'multi-family',
    'MR': 'multi-family',
    'OF': 'office',
    'ME': 'office', 
    'RS': 'retail', 
    'RB': 'retail',
    'MT': 'retail',
    'IW': 'industrial',
    'IL': 'industrial',
    'IH': 'industrial',
    'PA2': 'parking',
    'SC': 'school',
    'HO': 'hotel',
}

def tally_blg_by_type_cnty_dist(buildings, cnty_sd_crosswalk, building_type_dict):
    # 
    buildings_CntySD = pd.merge(
        left     = buildings,
        right    = cnty_sd_crosswalk,
        on       = 'parcel_id',
        how      = 'left',
        validate = 'many_to_one'
    )
    # summary
    buildings_CntySD['building_cat'] = buildings_CntySD['building_type'].map(building_type_dict)
    blgs_by_cnty_type = buildings_CntySD.groupby(
        ['county', 'superdistrict', 'building_cat'])[['residential_units', 'non_residential_sqft']].sum()
    blgs_by_cnty_type.reset_index(inplace=True)
    return blgs_by_cnty_type

In [5]:
NP_2020_blgs_by_cnty_type = tally_blg_by_type_cnty_dist(blg_2020_NP, parcel_taz_sd_crosswalk_df, building_type_dict)
print(len(NP_2020_blgs_by_cnty_type))
NP_2025_blgs_by_cnty_type = tally_blg_by_type_cnty_dist(blg_2025_NP, parcel_taz_sd_crosswalk_df, building_type_dict)
print(len(NP_2025_blgs_by_cnty_type))
FBP_2050_blgs_by_cnty_type = tally_blg_by_type_cnty_dist(blg_2050_FBP, parcel_taz_sd_crosswalk_df, building_type_dict)
print(len(FBP_2050_blgs_by_cnty_type))

299
299
300


In [6]:
NP_baseyear_blgs_by_cnty_type = pd.merge(
    NP_2020_blgs_by_cnty_type,
    NP_2025_blgs_by_cnty_type,
    on = ['county', 'superdistrict', 'building_cat'],
    how= 'outer',
    suffixes=('_2020', '_2025')
)
print(len(NP_baseyear_blgs_by_cnty_type))

NP_baseyear_blgs_by_cnty_type['residential_units_2023'] = \
    NP_baseyear_blgs_by_cnty_type['residential_units_2020'] + \
        ((2023-2020) / (2025-2020)) * (
            NP_baseyear_blgs_by_cnty_type['residential_units_2025'] - NP_baseyear_blgs_by_cnty_type['residential_units_2020'])
NP_baseyear_blgs_by_cnty_type['non_residential_sqft_2023'] = \
    NP_baseyear_blgs_by_cnty_type['non_residential_sqft_2020'] + \
        ((2023-2020) / (2025-2020)) * (
            NP_baseyear_blgs_by_cnty_type['non_residential_sqft_2025'] - NP_baseyear_blgs_by_cnty_type['non_residential_sqft_2020'])
display(NP_baseyear_blgs_by_cnty_type)

299


,county,superdistrict,building_cat,residential_units_2020,non_residential_sqft_2020,residential_units_2025,non_residential_sqft_2025,residential_units_2023,non_residential_sqft_2023
0,Alameda,9.0,single-family,64.0,0.0,64.0,0.0,64.0,0.0
1,Alameda,12.0,multi-family,5.0,0.0,5.0,0.0,5.0,0.0
2,Alameda,15.0,hotel,0.0,2382015.0,0.0,2382015.0,0.0,2382015.0
3,Alameda,15.0,industrial,54.0,25895630.0,54.0,26592729.0,54.0,26313889.4
4,Alameda,15.0,multi-family,17449.0,1548159.0,20212.0,1555679.0,19106.8,1552671.0
...,...,...,...,...,...,...,...,...,...
294,Sonoma,31.0,multi-family,5729.0,494259.0,6904.0,502459.0,6434.0,499179.0
295,Sonoma,31.0,office,35.0,9615980.0,35.0,9974616.0,35.0,9831161.6
296,Sonoma,31.0,retail,11.0,2896526.0,11.0,3139837.0,11.0,3042512.6
297,Sonoma,31.0,school,0.0,42960.0,0.0,42960.0,0.0,42960.0


In [7]:
blgs_by_cnty_type = pd.merge(
    NP_baseyear_blgs_by_cnty_type,
    FBP_2050_blgs_by_cnty_type.rename(
        columns={
            'residential_units': 'residential_units_2050',
            'non_residential_sqft': 'non_residential_sqft_2050'
        }
    ),
    on = ['county', 'superdistrict', 'building_cat'],
    how= 'outer',
)
print(len(blgs_by_cnty_type))


300


In [8]:
for col in list(blgs_by_cnty_type):
    if col not in ['county', 'superdistrict', 'building_cat']:
        blgs_by_cnty_type[col] = blgs_by_cnty_type[col].fillna(0)
blgs_by_cnty_type['resUnits_growth'] = blgs_by_cnty_type['residential_units_2050'] - blgs_by_cnty_type['residential_units_2023']
blgs_by_cnty_type['nonResSQFT_growth'] = blgs_by_cnty_type['non_residential_sqft_2050'] - blgs_by_cnty_type['non_residential_sqft_2023']
blgs_by_cnty_type

,county,superdistrict,building_cat,residential_units_2020,non_residential_sqft_2020,residential_units_2025,non_residential_sqft_2025,residential_units_2023,non_residential_sqft_2023,residential_units_2050,non_residential_sqft_2050,resUnits_growth,nonResSQFT_growth
0,Alameda,9.0,single-family,64.0,0.0,64.0,0.0,64.0,0.0,64.0,0.000000e+00,0.0,0.000000
1,Alameda,12.0,multi-family,5.0,0.0,5.0,0.0,5.0,0.0,5.0,0.000000e+00,0.0,0.000000
2,Alameda,15.0,hotel,0.0,2382015.0,0.0,2382015.0,0.0,2382015.0,0.0,2.382015e+06,0.0,0.000000
3,Alameda,15.0,industrial,54.0,25895630.0,54.0,26592729.0,54.0,26313889.4,54.0,2.622285e+07,0.0,-91038.400000
4,Alameda,15.0,multi-family,17449.0,1548159.0,20212.0,1555679.0,19106.8,1552671.0,56269.0,2.240918e+06,37162.2,688247.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Sonoma,31.0,office,35.0,9615980.0,35.0,9974616.0,35.0,9831161.6,35.0,9.907723e+06,0.0,76561.400000
296,Sonoma,31.0,retail,11.0,2896526.0,11.0,3139837.0,11.0,3042512.6,11.0,3.089121e+06,0.0,46608.400000
297,Sonoma,31.0,school,0.0,42960.0,0.0,42960.0,0.0,42960.0,0.0,4.296000e+04,0.0,0.000000
298,Sonoma,31.0,single-family,34545.0,451070.0,35414.0,449173.0,35066.4,449931.8,36099.0,4.491730e+05,1032.6,-758.800000


In [10]:
blgs_by_cnty_type.groupby(['county','building_cat'])[['resUnits_growth', 'nonResSQFT_growth']].sum().reset_index()

,county,building_cat,resUnits_growth,nonResSQFT_growth
0,Alameda,hotel,0.0,-6.540660e+04
1,Alameda,industrial,-2.0,-4.575307e+06
2,Alameda,multi-family,215820.4,-5.971164e+06
3,Alameda,office,0.0,-6.961256e+06
4,Alameda,parking,-962.6,0.000000e+00
...,...,...,...,...
65,Sonoma,office,0.0,-4.225902e+05
66,Sonoma,parking,0.0,0.000000e+00
67,Sonoma,retail,0.0,1.833647e+04
68,Sonoma,school,0.0,0.000000e+00


In [11]:
blgs_by_cnty_type.to_csv(r"C:\Users\ywang\Box\Plan Bay Area 2050+\Performance and Equity\Plan Performance\Equity_Performance_Metrics\Final_Blueprint\res_nonres_growth_by_cnty_type_FBP.csv", index=False)